In [17]:
import requests
import imageio.v3 as iio
import pickle
from sys import getsizeof
import numpy as np
from utils import preprocessing, uniform_sampling
import json
import cv2
import base64
from dask import delayed, compute

In [16]:
def resize_sequence(queue):
    batch = []
    for i in range(len(queue)):
        batch.append(delayed(cv2.resize)(queue[i], (224, 224)))
    
    return compute(*batch)

In [46]:
video_path = '/home/pep/Drive/PCLOUD/Projects/Best-Frame-Selection/0H2s9UJcNJ0_4_pdf.avi'

frames = iio.imread(video_path, extension='.avi')

# frames = np.load(video_path)['data']

# frames = uniform_sampling(frames)

# frames = preprocessing(frames.copy(), dynamic_crop=False).astype(np.float32)

frames_resize = np.stack(resize_sequence(frames)).astype(np.float32)

print(frames_resize.shape)

# arr = base64.b64encode(f.astype(np.float32).tobytes()).decode('utf-8')

arr = pickle.dumps(frames_resize)

print(getsizeof(arr))

url = ' http://54.179.5.58:80/predictions/fgn'

(64, 224, 224, 3)
38535367


In [ ]:
response = requests.post(
    url=url,
    data=arr)

In [11]:
response.content

b'{\n  "Fight": "0.98",\n  "NonFight": "0.02"\n}'

In [48]:
predict = json.loads(response.content.decode('utf-8'))
predict

{'code': 507, 'type': 'InternalServerException', 'message': 'Worker died.'}

In [10]:
import cv2
from collections import deque
import asyncio
import logging
from dask import delayed, compute

In [11]:
def resize_sequence(queue):
    batch = []
    for i in range(len(queue)):
        batch.append(delayed(cv2.resize)(queue[i], (224, 224)))
    
    return compute(*batch)

In [13]:
num_sequence = 64
url = 'http://localhost:8080/predictions/fgn'
queue = deque(maxlen=65)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    if len(queue) <= 0:
        for _ in range(num_sequence):
            queue.append(frame)
    
    queue.pop()
    queue.append(frame)
    
    send_queue = resize_sequence(queue)
    
    frames = np.stack(send_queue).astype(np.float32)
    
    data = pickle.dumps(frames)
    
    response = requests.post(url, data)

    result = json.loads(response.content.decode('utf-8'))
    
    class_pred = max(result, key=result.get)
    
    show_frame = queue[-1].copy()
    
    show_frame = cv2.putText(show_frame, class_pred, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.imshow('Frame', show_frame)
    
    if cv2.waitKey(1) == 27: break

cap.release()
cv2.destroyAllWindows()

In [5]:
cap.release()
cv2.destroyAllWindows()